In [47]:
import requests
from bs4 import BeautifulSoup 
import re
import unicodedata
import pythainlp.util
from pythainlp.tokenize import word_tokenize
from pythainlp.util import find_keyword
from pythainlp.util import rank
#from pythainlp.summarize import extract_keywords
from pythainlp.summarize import summarize
import itertools
import sqlite3
from urllib.parse import urljoin
import time
import string

In [89]:
class spyder:
    def __init__( self ,target_links,base_url ):
        self.base_url = base_url
        self.target_links = target_links
    
    def get_crawler(self):
        self.result_crawler = self.crawl(self.base_url,0,set())
        return self.result_crawler
    
    def get_check_domain(self):
        self.check_domain_result = self.check_domain(self.base_url,self.get_crawler())
        return self.check_domain_result
    
    def get_check_not_domain(self):
        self.check_not_domain_result = self.check_not_domain(self.base_url,self.get_crawler())   
        return self.check_not_domain_result
    
    def get_check_ref(self):
        self.check_ref_result = self.check_ref(self.get_check_not_domain(),self.target_links)
        return self.check_ref_result
    
    def get_all(self):
        crawl = self.crawl(self.base_url,0,set())
        check_domain =  self.check_domain(self.base_url,crawl) 
        check_not_domain = self.check_not_domain(self.base_url,crawl)
        check_ref = self.check_ref(check_not_domain,self.target_links)
        return crawl,check_domain,check_not_domain,check_ref
    
    def crawl(self,url, depth,visited):
        if depth < 2 :
            visited.add(url)
            headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
            time.sleep(0.3)
            response = requests.get(url,headers=headers)
            try:
                soup = BeautifulSoup(response.text, 'html.parser')
            except:
                soup = BeautifulSoup(response.text, 'lxml')
            links = soup.find_all('a')
            links = [link.get('href') for link in links if link.get('href') and not link.get('href').startswith('#')]
            links = [urljoin(url, link) for link in links if link]

            for link in links:
                if link not in visited:
                    link = link.replace(' ','')
                    visited.add(link)
                    #if link.startswith(url):
                    self.crawl(link,depth=depth+1, visited=visited)
        return visited
    
    def check_domain(self,base_url,links):
        result= set()
        for link in links :
            if link.startswith(base_url):
                result.add(link)
        return result
    
    def check_not_domain(self,base_url,links):
        result= set()
        for link in links :
            if not link.startswith(base_url):
                result.add(link)
        return result
    
    def check_ref(self,links,target_links):
        for i in links:
            for j in target_links:
                if i.startswith(j):
                    target_links[j]+=1
        return target_links

In [69]:
class Thai:
    def __init__(self,data:list):
        self.data_value = data
        self.sentence = self.get_sentence()
        self.keyword = self.get_keyword()
        self.summarize = self.get_summarize()
    def make_sentence(self,list_word):
        self.sentence_value = ''
        for i in list_word:
            for i in list_word:
                if pythainlp.util.countthai(i)<10:
                    list_word.remove(i)
        self.sentence_value = ' '.join(list_word)
        return self.sentence_value
    def get_sentence(self):
        self.sentence_result = self.make_sentence(self.data_value)
        return self.sentence_result
    def get_keyword(self):
        self.keyword_result = {}
        self.keyword_value = word_tokenize(self.sentence, engine="newmm")
        self.keyword_dict = find_keyword(self.keyword_value)
        # Iterate over the keys in the dictionary
        for key in self.keyword_dict:
        # Check if the key is text (i.e., not a space or quotation mark)
            if key.isalpha():
            # If the key is text, add it to the new dictionary
                self.keyword_result[key] = self.keyword_dict[key]
        return self.keyword_result
    def get_summarize(self):
        self.summarize_result =[]
        self.summarize_result = summarize(self.sentence,n=5)
        return self.summarize_result


In [83]:
def scrape_tags(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  try:
    title_tag = soup.find('title').text
  except:
    title_tag = soup.find('title')
  p_tags = soup.find_all('p')
  p_list =[]
  for p in p_tags:
    if p.string != None:
      p_list.append(unicodedata.normalize("NFKD", p.string))
  if len(p_list) == 0:
    p_list.append('ไม่พบข้อความในเว็บนี้')
  
  p_tag = "".join(p_list)
  thai_nlp = Thai(p_list)
  keyword = thai_nlp.keyword
  keyword = {k: v for k, v in sorted(keyword.items(), key=lambda item: item[1], reverse=True)}
  keyword = dict(itertools.islice(keyword.items(), 5))
  summarize_article = thai_nlp.summarize
  try:
    location = 'จ.'+max(thai_nlp.get_location().keys())
  except:
    location = 'NONE'
  
  if  title_tag == None:
    try:
      title_tag = summarize_article[0]
    except:
      title_tag = 'NONE'
    
  return p_tag, title_tag, keyword,location

In [96]:
target_links = {'https://www.thairath.co.th':0,'https://news.kapook.com':0}
doc=[]
for i in target_links:
    get_link = spyder(target_links,i)
    ref = get_link.get_check_ref()



for i in target_links:
    web_spyder=spyder(target_links,i)
    links,domain_links,not_domain_links,refer =web_spyder.get_all()
    for j in domain_links:
        j.replace(" ", "")
        d=dict()    
        body, title, keyword,location = scrape_tags(j)
        d['link']= j
        d['title'] = title
        d['body']=body
        d['location']=location
        d['keyword'] = keyword
        doc.append(d)


for i in doc:
    for j in ref:
        if i['link'].startswith(j):
            i['ref'] = ref[j]
print(doc)
        

{'https://www.thairath.co.th': 2, 'https://news.kapook.com': 0}
{'https://www.thairath.co.th/scoop/interview', 'https://www.thairath.co.th/tv/program', 'https://www.thairath.co.th/multimedia/gallery/lifestyle/travel/33851', 'https://www.thairath.co.th/multimedia/gallery', 'https://www.thairath.co.th/terms-condition', 'https://www.thairath.co.th/podcast/program/moohost', 'https://www.thairath.co.th', 'https://www.thairath.co.th/news/society/2630778', 'https://www.thairath.co.th/multimedia/gallery/sport/worldcup/34208', 'https://www.thairath.co.th/lifestyle/life/2630240', 'https://www.thairath.co.th/tv/program/thairath_news_show/723500', 'https://www.thairath.co.th/lottery', 'https://www.thairath.co.th/tv/watch', 'https://www.thairath.co.th/multimedia/gallery/entertain/news/34331', 'https://www.thairath.co.th/podcast/program/moohost/episode10', 'https://www.thairath.co.th/member/profile/following', 'https://www.thairath.co.th/privacypolicy', 'https://www.thairath.co.th/campaign/vote', 'h

In [92]:
import sqlite3

# Connect to SQLite3 database
conn = sqlite3.connect('inverted_index.db')

# Create tables for words, documents, and word frequencies
conn.execute('''
CREATE TABLE words (
    id INTEGER PRIMARY KEY,
    word TEXT NOT NULL UNIQUE
);
''')

conn.execute('''
CREATE TABLE documents (
    id INTEGER PRIMARY KEY,
    link TEXT NOT NULL UNIQUE,
    title TEXT,
    body TEXT
);
''')

conn.execute('''
CREATE TABLE word_frequencies (
    word_id INTEGER NOT NULL,
    doc_id INTEGER NOT NULL,
    frequency INTEGER NOT NULL,
    tfidf REAL NOT NULL,
    PRIMARY KEY (word_id, doc_id),
    FOREIGN KEY (word_id) REFERENCES words(id),
    FOREIGN KEY (doc_id) REFERENCES documents(id)
);
''')

KeyError: 0